# Digital Images and NumPy Arrays

*Dr Chas Nelson*

*Part of https://github.com/ChasNelson1990/image-processing-in-python*

## Objectives

* Understand how NumPy arrays can represent digital images
* Know how NumPy axes relates to image dimensions
* Learn how colour is represented by an adiditional axis in NumPy arrays 
* Understand how spatial and intensity sampling of an image relate to array size and `dtype` of NumPY arrays

## What is a Digital Image?

* Digital images are arrays or matrices of pixels (picture elements), i.e. numbers, that represent an object or scene.
  * Unless otherwise states we will assume these numbers are integers.
* In a grayscale image, each of these numbers is the intensity.
* A grayscale image usually represents a narrow band of wavelengths, i.e. a colour or channel.
  * Multiple grayscale images together can form a colour image, e.g. RGB.

![Digital images are arrays or matrices of pixels (or voxels) - i.e. numbers.](./assets/digitalimage.png)

*Courtesy of Dominic Waithe; apple from https://www.wikipiedia.org*

* We can navigate a 2D image by using row and column numbers to extract a single pixel value.
  * People use two coordinate systems for this: $i$ & $j$ or $x$ and $y$.
  * We will use $x$ for rows and $y$ for columns throughout this course. $x=0, y=0$ is the top left corner of the image.
  * (We will introduce higher dimensions as and when necessary.)
  
![Acessing pixels using axes.](./assets/arrays.png)

*Adapted from https://github.com/elegant-scipy/elegant-scipy*

## What is a NumPy Array?

* In Python, NumPy Arrays are arrays or matrices of numbers, as such we can use arrays to represent digital images.
* For the representation of a grayscale image, we would use an array with as many rows and columns as the each of these numbers in the grayvalue.
* We can navigate an array by using row and column numbers to extract a single element - this is called indexing.
  * Don't forget that Python starts counting at $0$ and not at $1$
  * As with digital images, we will use $x$ for rows and $y$ for columns throughout this course. $x=0, y=0$ is the top left corner of the array.
  * Note: Python and FIJI swap $x$ and $y$, so the data will be transposed.

In [ ]:
import numpy as np

# Create an array of a Guassian disc with values between 0 and 8
# The array has ten rows and fitteen columns
# Don't worry too much about this code - it just creates a toy example

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
myGaussianDisc = (8*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ) )).astype('uint8')

display(myGaussianDisc)

In [ ]:
# Extract the element value at x=5 and y=6
x = 5
y = 4
display(myGaussianDisc[x,y])

In [ ]:
# Import two key plotting/display packages
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
f,axes = plt.subplots(1,2)
(aMPL, aSNS) = axes.flatten()

# Display array as grayscale image
# Note that this autoscales (like ImageJ).
# This doesn't affect the underlying array
aMPL.imshow(myGaussianDisc, cmap="gray")  # We discuss colourmaps (cmaps, LUTs) below
aMPL.set_axis_off()  # This turns axes off
aMPL.set_title("grayscale Image...")

# Display array as heatmap
# Note that this also autoscales.
sns.heatmap(myGaussianDisc,cmap="viridis",square=True,annot=True,ax=aSNS)
aSNS.set_ylim(0,10)  # Make sure we can see the full array
aSNS.invert_yaxis()  # Heatmap and imshow flipx, let's undo that
aSNS.set_ylabel('X')  # Note that x and y are swapped for display (compared to indexing and FIJI)
aSNS.set_xlabel('Y')  # Note that x and y are swapped for display (compared to indexing and FIJI)
aSNS.set_title("... as Heatmap")

plt.show()

## Colour Digital Images

* Colour images have multiple 'channels.
* In photography and monitors/screens three such channels represent RGB.
* In microscopy we can have any number of channels, each representing a different spectral band, i.e. a different fluorphore.

## 'Colour' NumPy Arrays?

* NumPy arrays can have more than two dimensions ($x$ and $y$).
* If you're doing a lot of computation, it's conventional to represent channels in the first dimension, as this is quicker for access.
  * $x$ and $y$ then become the second and third dimensions, respectively.
* However, `matplotlib` expects colour to be the third dimension.
  * $x$ and $y$ are then set to the first and second dimension, respecitvely.
* We will use `matplotlib`'s convention for 2D images with colour channels.

In [ ]:
# Create a 3-channel array of offset Gaussian discs with values  between 0 and 8 (exclusive)
# The array has ten rows and fifteen columns

rows = 10
columns = 15
x, y = np.meshgrid(np.linspace(-1,1,columns), np.linspace(-1,1,rows))
d = np.sqrt(x*x+y*y)
sigma = 0.5
disc = (8*np.exp(-( (d)**2 / ( 2.0 * sigma**2 ) ))).astype('uint')
myRGBArray = np.stack([disc,np.roll(disc,2,axis=0),np.roll(disc,2,axis=1)],axis=2)

print("Red:")
display(myRGBArray[:,:,1])

In [ ]:
f, axes = plt.subplots(2,2)  # Create four subplots (2x2 grid)
(aR, aG, aB, aRGB) = axes.flatten()

aR.imshow(myRGBArray[:,:,0], cmap="gray")  # Displaying individual channels in gray prevents false highlighting of areas
aR.set_axis_off()
aR.set_title("Red Channel")

aG.imshow(myRGBArray[:,:,1], cmap="gray")
aG.set_axis_off()
aG.set_title("Green Channel")

aB.imshow(myRGBArray[:,:,2], cmap="gray")
aB.set_axis_off()
aB.set_title("Blue Channel")

aRGB.imshow(myRGBArray/8)  # Matlab expects values between 0 and 1 or 0 and 255 (see bit-depth below)
aRGB.set_axis_off()
aRGB.set_title("Composite")

plt.show()

## Spatial Sampling

* The number of pixels in in a image denote the resolution.
  * In NumPy, the resolution is the shape of the array, e.g. `myArray.shape`
* Each pixel will represent a physical size, e.g. $3 \mu m \times 3 \mu m$ (also often call resolution)
  * This is not explicitly coded into a digital image or NumPy array so we will need to be extracted from the metadata.

<!-- ![The number of pixels in in a image denote the resolution.](./assets/resolution.png) -->

## Intensity Sampling

* The bit-depth of an image describes the dynamic range of a pixel, i.e. the difference between the minimum and maximum possible values.
  * The equivalent feature of a Numpy array is its `dtype`.
* Most DSLR cameras will use 8-bit for grayscale images and 3x8-bit (24-bit) for colour images.
  * In NumPy this is `uint8`.
* Most scientific cameras will use 12-bit or 16-bit. (Both will appear to be 16-bit due to the container files.)
  * In NumPy this is `uint16`.
* A lot of image processing requires continuous pixel values, i.e. decimals between 0 and 1.
  * In NumPy these values are `float`.
* Up to 16-bit, computers use unsigned integers to represent pixel values; however, 32 and 64-bit images will likely use `float` (continous numbers with decimal points). Each has its benefits.
* It is always possible to process images in a higher bit-depth container/array without lose of data, e.g. converting an 8-bit image to a `float` array; but be wary of accidentally doing things the other way, e.g. opening a 16-bit image in an `uint8` array.

![The bit-depth of an image describes the dynamic range of a pixel, i.e. the difference between the minimum and maximum pixel values.](./assets/bitdepth.png)

*Courtesy of Dominic Waithe; apple from https://www.wikipedia.org*

## Key Points

* `numpy` arrays can be used to store and manipulate digital images
* An array element is equivalent to a pixel in a digital image
* Each array axis represents a dimension of the image, e.g. in a 2D greyscale image axis 0 is x and axis 1 is y
* A third axis can include colour/spectral information
* The `shape` of a NumPy array corresponds to the image resolution, i.e. the number of pixels in the image
* However, spatial sampling, e.g. how big a pixel is, is not intrinsically included in a NumPy array
* Intensity sampling, e.g. bit-depth, of an image maps to the `dtype` of NumPY arrays, e.g. 8-bit maps to 'uint8' and 16-bit to 'uint16'

## Any Bugs/Issues/Comments?

If you've found a bug or have any comments about this notebook, please fill out this on-line form:

Any feedback I get I will try to correct/implement as soon as possible.